In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import time
import os

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose(
    [
     transforms.RandomHorizontalFlip(),
     transforms.RandomGrayscale(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform1 = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform1)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
class Net(nn.Module):
    def __init__(self):
        """
        describe:基于VGG16，提出输入224x224x3，所以这里会出现维度的奇偶不匹配
        """
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.conv5 = nn.Conv2d(128,128, 3,padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv7 = nn.Conv2d(128, 128, 1,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

        self.conv8 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv9 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, 1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()

        self.conv11 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, 1, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        self.fc14 = nn.Linear(512*4*4,1024)
        self.drop1 = nn.Dropout2d()
        self.fc15 = nn.Linear(1024,1024)
        self.drop2 = nn.Dropout2d()
        self.fc16 = nn.Linear(1024,10)


    def forward(self,x):
        """
        describe:
        input(100,3,32,32) -> conv1 (100,64,32,32)-> conv2 -> pool1(100,64,16,16) -> 
        conv3(100,128,16,16) -> conv4 -> pool2(100,128,9,9) -> 
        conv5 -> conv6 -> conv7(100,128,11,11) -> maxpool3(100,128,6,6) -> 
        conv8(100,256,6,6) -> conv9 -> conv10(100,256,8,8) -> pool4(100,256,5,5) ->
        conv11(100,512,5,5) -> conv12 -> conv13 (100,512,7,7)-> pool5(100,512,4,4) -> 
        reshape(100,512x4x4) -> fc14(100,1024) -> fc15 -> fc16(100,10) -> output
        """
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        # print(" x shape ",x.size())
        x = x.view(-1,512*4*4)
        x = F.relu(self.fc14(x))
        x = self.drop1(x)
        x = F.relu(self.fc15(x))
        x = self.drop2(x)
        x = self.fc16(x)

        return x

In [19]:
net = Net().to(device)
print(net)
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(self.parameters(),lr=0.01)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=1, dila

In [15]:
total_step = len(trainloader)
for epoch in range(100):  # loop over the dataset multiple times
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = net(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print("Epoch: [{}/100], Step: [{}/{}], Loss: {:.4f}".format(
                    epoch+1, i+1, total_step, loss.item()))

Epoch: [1/100], Step: [100/500], Loss: 1.6203
Epoch: [1/100], Step: [200/500], Loss: 1.4013
Epoch: [1/100], Step: [300/500], Loss: 1.4038
Epoch: [1/100], Step: [400/500], Loss: 1.0145
Epoch: [1/100], Step: [500/500], Loss: 0.8920
Epoch: [2/100], Step: [100/500], Loss: 1.2093
Epoch: [2/100], Step: [200/500], Loss: 1.2170
Epoch: [2/100], Step: [300/500], Loss: 1.0429
Epoch: [2/100], Step: [400/500], Loss: 0.8567
Epoch: [2/100], Step: [500/500], Loss: 0.8855
Epoch: [3/100], Step: [100/500], Loss: 0.8823
Epoch: [3/100], Step: [200/500], Loss: 0.7478
Epoch: [3/100], Step: [300/500], Loss: 0.7270
Epoch: [3/100], Step: [400/500], Loss: 0.6535
Epoch: [3/100], Step: [500/500], Loss: 0.7517
Epoch: [4/100], Step: [100/500], Loss: 0.9052
Epoch: [4/100], Step: [200/500], Loss: 0.5992
Epoch: [4/100], Step: [300/500], Loss: 0.7525
Epoch: [4/100], Step: [400/500], Loss: 0.4795
Epoch: [4/100], Step: [500/500], Loss: 0.4614
Epoch: [5/100], Step: [100/500], Loss: 0.6441
Epoch: [5/100], Step: [200/500], L

KeyboardInterrupt: 

In [16]:
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the model on the test images: {}%'.format(100*correct/total))

Accuracy of the model on the test images: 83.86%


In [18]:
torch.save(net.state_dict(), 'VGG.ckpt')